In [1]:
import pandas as pd

import os

In [2]:
def caminho_absoluto(caminho_relativo_com_barras_normais):
    
    caminho_base = os.getcwd()

    caminho_absoluto = os.path.join(caminho_base, caminho_relativo_com_barras_normais)

    return caminho_absoluto

In [3]:
situacao_final = pd.read_csv(caminho_absoluto('data/tratamento_curva_abc/dados_tratados/csv/situacao_final.csv'))

In [4]:
situacao_final.head(5)

,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,...,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Frac.,Tipo,Qtde Desvio Picking,local
0,106011.0,NARIDRIN 12 HORAS FR C/30ML,A,A,A,303073.0,5486.0,851628.0,276.0,251.0,...,18,3.0,100.0,88.0,300.0,Não,29.913,Ponta De G,3878.0,pallet
1,231012.0,EXPEC XAROPE FR C/120ML,A,A,A,148274.0,5945.0,504974.0,276.0,219.0,...,62,1.0,60.0,1.0,240.0,Não,26.025,Flowrack,2397.0,pallet
2,102011.0,CYSTEX C/ 24DRG,A,A,A,116576.0,215.0,137979.0,289.0,52.0,...,0,15.0,100.0,270.0,300.0,Não,29.063,Flowrack,4361.0,apanha_a
3,101190.0,EMSEXPECTOR XPE 120ML,A,A,A,111966.0,867.0,163898.0,286.0,164.0,...,10,3.0,60.0,233.0,240.0,Não,21.911,Ponta De G,3046.0,pallet
4,231064.0,REPOFLOR 200MG C/ 06CAPS,A,A,A,107501.0,255.0,132993.0,275.0,51.0,...,7,15.0,100.0,196.0,300.0,Não,25.012,Flowrack,3683.0,apanha_a


In [5]:
selecao_dividir = ((situacao_final['Estoque Frac'] != 0) & \
                   (situacao_final['Tipo'] == 'Prateleira') & \
                   (situacao_final['Ender.Frac.'] > 20) & \
                   (situacao_final['Ender.Frac.'] < 28)
                   )

prateleira_para_dividir = situacao_final[selecao_dividir]

In [6]:
selecao_nao_e_end_do_meio = ~((prateleira_para_dividir['Ender.Frac.'] > 21.580) & \
                              (prateleira_para_dividir['Ender.Frac.'] < 22) | \
                              (prateleira_para_dividir['Ender.Frac.'] > 22.580) & \
                              (prateleira_para_dividir['Ender.Frac.'] < 23)
                              )

prateleira_para_dividir = prateleira_para_dividir[selecao_nao_e_end_do_meio]

In [7]:
def comeca_com(df, corredor):
    return df['Ender.Frac.'].astype(str).str.startswith(corredor)

In [8]:
# Mais saida
ir_para_final_modulo = prateleira_para_dividir[0:768]

selecao_trocar_para_o_final = ((comeca_com(ir_para_final_modulo, '20')) | \
                               (comeca_com(ir_para_final_modulo, '23')) | \
                               (comeca_com(ir_para_final_modulo, '24')) | \
                               (comeca_com(ir_para_final_modulo, '27'))
                               )

colocar_nos_corredores_21_22_25_26 = ir_para_final_modulo[selecao_trocar_para_o_final]

In [9]:
print(len(colocar_nos_corredores_21_22_25_26))
colocar_nos_corredores_21_22_25_26.head(3)

393


,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,...,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Frac.,Tipo,Qtde Desvio Picking,local
47,63591.0,"COMPRESSA HERIKA 13 FIOS 7,5CMX7,5 C/10",A,C,A,21903.0,1.0,22603.0,38.0,1.0,...,1 (+488 fr),2.0,700.0,50.0,50.0,Sim,24.144,Prateleira,86.0,apanha_b
132,690069.0,CIMEGRIPE C/ 20CAPS,A,A,A,7647.0,27.0,10887.0,116.0,16.0,...,0,5.0,120.0,1.0,30.0,Sim,24.327,Prateleira,121.0,apanha_a
147,64380.0,"COMPRESSA HÉRIKA SOFT 13 FIOS/CM2,7,5CM",A,C,A,7012.0,0.0,7012.0,13.0,0.0,...,1,1.0,840.0,418.0,20.0,Não,20.224,Prateleira,84.0,apanha_b


In [10]:
colocar_nos_corredores_21_22_25_26.to_excel('colocar_nos_corredores_21_22_25_26.xlsx', index=False)

In [11]:
ir_para_comeco_modulo = prateleira_para_dividir[768:]

selecao_trocar_para_o_comeco = ((comeca_com(ir_para_comeco_modulo, '21')) | \
                                (comeca_com(ir_para_comeco_modulo, '22')) | \
                                (comeca_com(ir_para_comeco_modulo, '25')) | \
                                (comeca_com(ir_para_comeco_modulo, '26'))
                                )

colocar_nos_corredores_20_23_24_27 = ir_para_comeco_modulo[selecao_trocar_para_o_comeco]

In [12]:
print(len(colocar_nos_corredores_20_23_24_27))
colocar_nos_corredores_20_23_24_27.head(3)

250


,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,...,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Frac.,Tipo,Qtde Desvio Picking,local
1970,610153.0,TANDRIFLAN C/ 15COMP,C,NaN,C,116.0,NaN,116.0,33.0,NaN,...,0 (+196 fr),1.0,198.0,6.0,45.0,Sim,22.495,Prateleira,6.0,apanha_c
1976,1006492.0,LORATADINA 10MG C/ 12COMP,C,C,B,115.0,2.0,235.0,5.0,2.0,...,1,1.0,60.0,65.0,60.0,Não,22.494,Prateleira,7.0,apanha_c
1977,29020.0,BISSULFATO CLOPIDOGREL 75MG C/ 28COMP,C,NaN,C,115.0,NaN,115.0,15.0,NaN,...,0,1.0,56.0,35.0,56.0,Não,26.456,Prateleira,1.0,apanha_c


In [13]:
colocar_nos_corredores_20_23_24_27.to_excel('colocar_nos_corredores_20_23_24_27.xlsx', index=False)

In [ ]:
situacao_final_ = pd.read_csv(caminho_absoluto('data/tratamento_curva_abc/dados_tratados/csv/situacao_final.csv'))

selecao_dividir = ((situacao_final_['Estoque Frac'] != 0) & \
                   (situacao_final_['Tipo'] == 'Prateleira') & \
                   (situacao_final_['Ender.Frac.'] > 20) & \
                   (situacao_final_['Ender.Frac.'] < 28)
                   )

prateleira_para_dividir = situacao_final_[selecao_dividir]

selecao_nao_e_end_do_meio = ~((prateleira_para_dividir['Ender.Frac.'] > 21.580) & \
                              (prateleira_para_dividir['Ender.Frac.'] < 22) | \
                              (prateleira_para_dividir['Ender.Frac.'] > 22.580) & \
                              (prateleira_para_dividir['Ender.Frac.'] < 23)
                              )

prateleira_para_dividir = prateleira_para_dividir[selecao_nao_e_end_do_meio]

def comeca_com(df, corredor):
    return df['Ender.Frac.'].astype(str).str.startswith(corredor)

# Mais saida
ir_para_final_modulo = prateleira_para_dividir[0:768]

selecao_trocar_para_o_final = ((comeca_com(ir_para_final_modulo, '20')) | \
                               (comeca_com(ir_para_final_modulo, '23')) | \
                               (comeca_com(ir_para_final_modulo, '24')) | \
                               (comeca_com(ir_para_final_modulo, '27'))
                               )

colocar_nos_corredores_21_22_25_26 = ir_para_final_modulo[selecao_trocar_para_o_final]

ir_para_comeco_modulo = prateleira_para_dividir[768:]

selecao_trocar_para_o_comeco = ((comeca_com(ir_para_comeco_modulo, '21')) | \
                                (comeca_com(ir_para_comeco_modulo, '22')) | \
                                (comeca_com(ir_para_comeco_modulo, '25')) | \
                                (comeca_com(ir_para_comeco_modulo, '26'))
                                )

colocar_nos_corredores_20_23_24_27 = ir_para_comeco_modulo[selecao_trocar_para_o_comeco]